## 1. Configuración del Entorno

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import os
import gc
import json
import warnings
import cv2
from tqdm import tqdm
import zipfile
import shutil

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import (confusion_matrix, roc_auc_score, roc_curve, 
                           matthews_corrcoef, balanced_accuracy_score,
                           precision_recall_curve, average_precision_score,
                           f1_score, recall_score, precision_score)
from sklearn.utils import class_weight

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.callbacks import (ModelCheckpoint, EarlyStopping, 
                                       ReduceLROnPlateau, TensorBoard, Callback)

warnings.filterwarnings('ignore')

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"GPU configurada: {gpus}")
    except RuntimeError as e:
        print(e)
else:
    print("No se detectó ninguna GPU, usando CPU")


MOBILE_IMG_SIZE = 224
BATCH_SIZE = 128
EPOCHS = 30 
TARGET_DATASET_SIZE = 20000  
MELANOMA_RATIO = 0.05  

SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {tf.keras.__version__}")
print(f"Configuración completada ✓")

## 2. Carga y Preparación del Dataset

In [ ]:
BASE_PATH = Path('/kaggle/input/siim-isic-melanoma-classification')
train_df = pd.read_csv(BASE_PATH / 'train.csv')
train_df['image_path'] = train_df['image_name'].apply(
    lambda x: str(BASE_PATH / 'jpeg' / 'train' / f'{x}.jpg')
)

print("Análisis del Dataset Original:")
print(f"Total imágenes: {len(train_df):,}")
print(f"Melanomas: {len(train_df[train_df['target']==1]):,} ({train_df['target'].mean()*100:.2f}%)")
print(f"\nDistribución por características:")
print(f"- Edad promedio: {train_df['age_approx'].mean():.1f} años")
print(f"- Distribución por sexo: {train_df['sex'].value_counts().to_dict()}")
print(f"- Sitios anatómicos: {train_df['anatom_site_general_challenge'].value_counts().head()}")

In [ ]:
def smart_subsample_optimized(df, target_total=20000, melanoma_ratio=0.05, seed=42):
    np.random.seed(seed)
    
    melanomas = df[df['target'] == 1].copy()
    benignos = df[df['target'] == 0].copy()
    
    n_melanomas = len(melanomas)
    n_benignos_needed = min(
        int(n_melanomas / melanoma_ratio - n_melanomas),
        target_total - n_melanomas,
        len(benignos)
    )
    
    benignos['age_group'] = pd.cut(
        benignos['age_approx'].fillna(benignos['age_approx'].median()),
        bins=[0, 20, 40, 60, 80, 100],
        labels=['0-20', '20-40', '40-60', '60-80', '80+']
    )
    benignos['anatom_site_general_challenge'].fillna('unknown', inplace=True)
    benignos['sex'].fillna('unknown', inplace=True)
    
    stratify_cols = ['age_group', 'anatom_site_general_challenge', 'sex']
    
    benignos['strat_group'] = benignos[stratify_cols].astype(str).agg('_'.join, axis=1)
    
    group_sizes = benignos['strat_group'].value_counts()
    samples_per_group = (group_sizes / len(benignos) * n_benignos_needed).round().astype(int)
    
    sampled_benignos = []
    for group, n_samples in samples_per_group.items():
        if n_samples > 0:
            group_df = benignos[benignos['strat_group'] == group]
            n_samples = min(n_samples, len(group_df))
            sampled = group_df.sample(n=n_samples, random_state=seed)
            sampled_benignos.append(sampled)
    
    benignos_sampled = pd.concat(sampled_benignos)
    
    if len(benignos_sampled) > n_benignos_needed:
        benignos_sampled = benignos_sampled.sample(n=n_benignos_needed, random_state=seed)
    elif len(benignos_sampled) < n_benignos_needed:

        remaining = n_benignos_needed - len(benignos_sampled)
        additional = benignos[~benignos.index.isin(benignos_sampled.index)].sample(
            n=min(remaining, len(benignos) - len(benignos_sampled)), 
            random_state=seed
        )
        benignos_sampled = pd.concat([benignos_sampled, additional])
    
    balanced_df = pd.concat([melanomas, benignos_sampled]).sample(frac=1, random_state=seed).reset_index(drop=True)
    
    print(f"\nDataset Modificado:")
    print(f"  - Total: {len(balanced_df):,} imágenes")
    print(f"  - Melanomas: {len(melanomas):,} ({len(melanomas)/len(balanced_df)*100:.1f}%)")
    print(f"  - Benignos: {len(benignos_sampled):,} ({len(benignos_sampled)/len(balanced_df)*100:.1f}%)")
    print(f"  - Ratio real: 1:{len(benignos_sampled)/len(melanomas):.1f}")
    
    return balanced_df

train_df_balanced = smart_subsample_optimized(
    train_df, 
    target_total=TARGET_DATASET_SIZE, 
    melanoma_ratio=MELANOMA_RATIO
)

## 3. División de Datos y Análisis de la Distribución

In [ ]:

train_data, temp_data = train_test_split(
    train_df_balanced,
    test_size=0.3,
    stratify=train_df_balanced['target'],
    random_state=SEED
)

val_data, test_data = train_test_split(
    temp_data,
    test_size=0.5,
    stratify=temp_data['target'],
    random_state=SEED
)

print(f"División de datos:")
print(f"  - Entrenamiento: {len(train_data):,} ({len(train_data)/len(train_df_balanced)*100:.1f}%)")
print(f"  - Validación: {len(val_data):,} ({len(val_data)/len(train_df_balanced)*100:.1f}%)")
print(f"  - Prueba: {len(test_data):,} ({len(test_data)/len(train_df_balanced)*100:.1f}%)")

class_weights_array = class_weight.compute_class_weight(
    'balanced',
    classes=np.unique(train_data['target']),
    y=train_data['target']
)

CLASS_WEIGHTS = {
    0: 1.0,
    1: class_weights_array[1] * 3.0
}

print(f"\nPesos de clase que se han calculado: {CLASS_WEIGHTS}")

## 4. Aumentación de Datos

In [ ]:

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=180,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.8, 1.2],
    channel_shift_range=20,
    shear_range=10,
    fill_mode='reflect',
    preprocessing_function=None
)

val_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    zoom_range=0.05,
    brightness_range=[0.95, 1.05]
)

train_generator = train_datagen.flow_from_dataframe(
    train_data,
    x_col='image_path',
    y_col='target',
    target_size=(MOBILE_IMG_SIZE, MOBILE_IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='raw',
    shuffle=True,
    seed=SEED
)

val_generator = val_datagen.flow_from_dataframe(
    val_data,
    x_col='image_path',
    y_col='target',
    target_size=(MOBILE_IMG_SIZE, MOBILE_IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='raw',
    shuffle=False
)

test_generator = test_datagen.flow_from_dataframe(
    test_data,
    x_col='image_path',
    y_col='target',
    target_size=(MOBILE_IMG_SIZE, MOBILE_IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='raw',
    shuffle=False
)

print(f"Pasos en entrenamiento: {len(train_generator)}")
print(f"Pasos en validación: {len(val_generator)}")
print(f"Pasos en prueba: {len(test_generator)}")

## 5. Focal Loss y Métricas Customizadas

In [9]:
def focal_loss(gamma=2.0, alpha=0.75):

    def focal_loss_fixed(y_true, y_pred):
        epsilon = tf.keras.backend.epsilon()
        y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)
        
        y_true = tf.cast(y_true, tf.float32)
        
        p_t = tf.where(tf.equal(y_true, 1), y_pred, 1 - y_pred)
        alpha_factor = tf.where(tf.equal(y_true, 1), alpha, 1 - alpha)
        
        cross_entropy = -tf.math.log(p_t)
        weight = alpha_factor * tf.pow((1 - p_t), gamma)
        
        loss = weight * cross_entropy
        return tf.reduce_mean(loss)
    
    return focal_loss_fixed

class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.precision = tf.keras.metrics.Precision()
        self.recall = tf.keras.metrics.Recall()
        
    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)
        
    def result(self):
        precision = self.precision.result()
        recall = self.recall.result()
        return 2 * ((precision * recall) / (precision + recall + tf.keras.backend.epsilon()))
    
    def reset_state(self):
        self.precision.reset_state()
        self.recall.reset_state()

class MedicalMetrics(Callback):
    def __init__(self, validation_data, threshold=0.5):
        super().__init__()
        self.validation_data = validation_data
        self.threshold = threshold
        
    def on_epoch_end(self, epoch, logs=None):

        val_generator = self.validation_data[0]
        val_labels = self.validation_data[1]
        
        val_generator.reset()
        predictions = self.model.predict(val_generator, verbose=0)
        y_pred = predictions.flatten()
        y_true = val_labels[:len(y_pred)]
        
        y_pred_binary = (y_pred > self.threshold).astype(int)
        
        mcc = matthews_corrcoef(y_true, y_pred_binary)
        
        balanced_acc = balanced_accuracy_score(y_true, y_pred_binary)
        
        tp = np.sum((y_true == 1) & (y_pred_binary == 1))
        fn = np.sum((y_true == 1) & (y_pred_binary == 0))
        sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
        
        tn = np.sum((y_true == 0) & (y_pred_binary == 0))
        fp = np.sum((y_true == 0) & (y_pred_binary == 1))
        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
        
        print(f"\n - MCC: {mcc:.4f} | Balanced Acc: {balanced_acc:.4f}")
        print(f" - Sensitivity: {sensitivity:.4f} | Specificity: {specificity:.4f}")

## 6. Arquitectura con EfficientNetV2 como base

In [ ]:
def build_efficientnetv2_model(
    img_size=MOBILE_IMG_SIZE, 
    num_classes=1,
    dropout_rate=0.3,
    l2_reg=1e-4
):
    inputs = layers.Input(shape=(img_size, img_size, 3), name='input')

    x = tf.keras.applications.efficientnet_v2.preprocess_input(inputs)

    base_model = tf.keras.applications.EfficientNetV2B0(
        input_shape=(img_size, img_size, 3),
        include_top=False,
        weights='imagenet',
        include_preprocessing=False
    )

    base_model.trainable = False

    x = base_model(x, training=False)

    gap = layers.GlobalAveragePooling2D(name='gap')(x)
    gmp = layers.GlobalMaxPooling2D(name='gmp')(x)

    concat = layers.Concatenate()([gap, gmp])
    
    x = layers.Dense(
        256, 
        activation='relu',
        kernel_regularizer=regularizers.l2(l2_reg),
        name='dense_1'
    )(concat)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(dropout_rate)(x)
    
    x = layers.Dense(
        128,
        activation='relu',
        kernel_regularizer=regularizers.l2(l2_reg),
        name='dense_2'
    )(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(dropout_rate * 0.7)(x)
    
    x = layers.Dense(
        64,
        activation='relu',
        kernel_regularizer=regularizers.l2(l2_reg),
        name='dense_3'
    )(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(dropout_rate * 0.5)(x)
    
    # Output layer
    outputs = layers.Dense(
        num_classes,
        activation='sigmoid',
        kernel_regularizer=regularizers.l2(l2_reg),
        name='output'
    )(x)
    
    model = models.Model(inputs=inputs, outputs=outputs, name='EfficientNetV2_Melanoma')
    
    return model, base_model

model, base_model = build_efficientnetv2_model()

print(f"Parámetros totales: {model.count_params():,}")
print(f"Parámetros entrenables: {sum([tf.size(w).numpy() for w in model.trainable_weights]):,}")

## 7. Compilación con Focal Loss y métricas

In [ ]:
initial_learning_rate = 0.001
lr_schedule = tf.keras.optimizers.schedules.CosineDecay(
    initial_learning_rate,
    decay_steps=len(train_generator) * EPOCHS,
    alpha=0.1
)

optimizer = tf.keras.optimizers.Adam(
    learning_rate=lr_schedule,
    clipnorm=1.0
)

model.compile(
    optimizer=optimizer,
    loss=focal_loss(gamma=2.0, alpha=0.75),
    metrics=[
        'accuracy',
        tf.keras.metrics.AUC(name='auc', curve='ROC'),
        tf.keras.metrics.AUC(name='pr_auc', curve='PR'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        F1Score(name='f1')
    ]
)

print("Modelo compilado con Focal Loss y métricas varias")

## 8. Callbacks y Entrenamiento

In [12]:
os.makedirs('checkpoints', exist_ok=True)
os.makedirs('logs', exist_ok=True)


callbacks = [

    ModelCheckpoint(
        'checkpoints/best_model_efficientnet.h5',
        monitor='val_recall',
        mode='max',
        save_best_only=True,
        save_weights_only=False,
        verbose=1
    ),
    
    ModelCheckpoint(
        'checkpoints/model_epoch_{epoch:02d}_valloss_{val_loss:.3f}.h5',
        monitor='val_loss',
        save_best_only=False,
        save_weights_only=False,
        verbose=0
    ),
    
    EarlyStopping(
        monitor='val_recall',
        patience=10,
        restore_best_weights=True,
        min_delta=0.001,
        verbose=1
    ),

    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=4,
        min_lr=1e-7,
        cooldown=2,
        verbose=1
    ),
    
    TensorBoard(
        log_dir='logs',
        histogram_freq=1,
        write_graph=True,
        update_freq='epoch',
        profile_batch=0
    ),
    
    MedicalMetrics(
        validation_data=(val_generator, val_data['target'].values),
        threshold=0.5
    )
]

In [ ]:
print("\n" + "="*60)
print("FASE 1: ENTRENAMIENTO INICIAL")
print("="*60)

history_phase1 = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=15,
    callbacks=callbacks,
    class_weight=CLASS_WEIGHTS,
    verbose=1
)

In [ ]:
print("\n" + "="*60)
print("FASE 2: FINE-TUNING")
print("="*60)

base_model.trainable = True
fine_tune_at = len(base_model.layers) - 50

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001, clipnorm=1.0),
    loss=focal_loss(gamma=2.0, alpha=0.75),
    metrics=[
        'accuracy',
        tf.keras.metrics.AUC(name='auc', curve='ROC'),
        tf.keras.metrics.AUC(name='pr_auc', curve='PR'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        F1Score(name='f1')
    ]
)

print(f"Capas entrenables: {len([l for l in model.layers if l.trainable])}")

history_phase2 = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    initial_epoch=15,
    callbacks=callbacks,
    class_weight=CLASS_WEIGHTS,
    verbose=1
)

## 9. Optimización del Umbral

In [ ]:
def find_optimal_threshold(model, generator, min_sensitivity=0.90):
    generator.reset()
    predictions = model.predict(generator, verbose=1)
    y_true = generator.labels[:len(predictions)]
    y_pred = predictions.flatten()
    
    fpr, tpr, thresholds = roc_curve(y_true, y_pred)

    valid_idx = np.where(tpr >= min_sensitivity)[0]
    if len(valid_idx) > 0:
        optimal_idx = valid_idx[0]
    else:
        optimal_idx = np.argmax(tpr - fpr)
    
    optimal_threshold = thresholds[optimal_idx]
    
    return optimal_threshold, fpr[optimal_idx], tpr[optimal_idx], y_true, y_pred

optimal_threshold, fpr_opt, tpr_opt, y_true, y_pred = find_optimal_threshold(
    model, val_generator, min_sensitivity=0.90
)

print(f"\nUmbral óptimo: {optimal_threshold:.4f}")
print(f"Sensibilidad: {tpr_opt:.2%}")
print(f"Especificidad: {1-fpr_opt:.2%}")

## 10. Evaluación Completa del Modelo

In [ ]:
def evaluate_model_comprehensive(model, test_generator, threshold=0.5):
    test_generator.reset()
    predictions = model.predict(test_generator, verbose=1)
    y_true = test_generator.labels[:len(predictions)]
    y_pred = predictions.flatten()
    y_pred_binary = (y_pred >= threshold).astype(int)

    cm = confusion_matrix(y_true, y_pred_binary)
    tn, fp, fn, tp = cm.ravel()

    metrics = {
        'threshold': threshold,
        'auc_roc': roc_auc_score(y_true, y_pred),
        'average_precision': average_precision_score(y_true, y_pred),
        'sensitivity': tp / (tp + fn) if (tp + fn) > 0 else 0,
        'specificity': tn / (tn + fp) if (tn + fp) > 0 else 0,
        'precision': tp / (tp + fp) if (tp + fp) > 0 else 0,
        'npv': tn / (tn + fn) if (tn + fn) > 0 else 0,
        'f1_score': f1_score(y_true, y_pred_binary),
        'mcc': matthews_corrcoef(y_true, y_pred_binary),
        'balanced_accuracy': balanced_accuracy_score(y_true, y_pred_binary),
        'nnd': 1 / (tp / (tp + fn)) if tp > 0 else float('inf'),
        'confusion_matrix': cm
    }
    
    # Imprimir resultados
    print("\n" + "="*60)
    print("EVALUACIÓN COMPLETA DEL MODELO")
    print("="*60)
    print(f"Umbral: {threshold:.4f}")
    print(f"\nMétricas de Rendimiento:")
    print(f"  - AUC-ROC: {metrics['auc_roc']:.4f}")
    print(f"  - Average Precision: {metrics['average_precision']:.4f}")
    print(f"  - Sensibilidad (Recall): {metrics['sensitivity']:.2%}")
    print(f"  - Especificidad: {metrics['specificity']:.2%}")
    print(f"  - Precisión: {metrics['precision']:.2%}")
    print(f"  - NPV: {metrics['npv']:.2%}")
    print(f"  - F1-Score: {metrics['f1_score']:.4f}")
    print(f"  - MCC: {metrics['mcc']:.4f}")
    print(f"  - Balanced Accuracy: {metrics['balanced_accuracy']:.2%}")
    print(f"  - NND: {metrics['nnd']:.2f}")
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(
        cm, 
        annot=True, 
        fmt='d', 
        cmap='Blues',
        xticklabels=['Benigno', 'Melanoma'],
        yticklabels=['Benigno', 'Melanoma'],
        cbar_kws={'label': 'Cantidad'}
    )
    plt.title(f'Matriz de Confusión\n(Umbral: {threshold:.3f})')
    plt.ylabel('Verdadero')
    plt.xlabel('Predicho')
    
    for i in range(2):
        for j in range(2):
            percentage = cm[i, j] / cm.sum() * 100
            plt.text(j + 0.5, i + 0.7, f'({percentage:.1f}%)', 
                    ha='center', va='center', fontsize=9, color='gray')
    
    plt.tight_layout()
    plt.savefig('confusion_matrix_optimized.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    return metrics

test_metrics = evaluate_model_comprehensive(model, test_generator, optimal_threshold)

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

ax1 = axes[0, 0]
fpr, tpr, _ = roc_curve(y_true, y_pred)
ax1.plot(fpr, tpr, 'b-', lw=2, label=f'AUC = {test_metrics["auc_roc"]:.3f}')
ax1.plot([0, 1], [0, 1], 'k--', lw=1)
ax1.scatter(fpr_opt, tpr_opt, c='red', s=100, marker='o', 
           label=f'Umbral óptimo = {optimal_threshold:.3f}')
ax1.set_xlabel('Tasa de Falsos Positivos')
ax1.set_ylabel('Tasa de Verdaderos Positivos')
ax1.set_title('Curva ROC')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2 = axes[0, 1]
precision, recall, _ = precision_recall_curve(y_true, y_pred)
ax2.plot(recall, precision, 'g-', lw=2, 
        label=f'AP = {test_metrics["average_precision"]:.3f}')
ax2.set_xlabel('Recall (Sensibilidad)')
ax2.set_ylabel('Precisión')
ax2.set_title('Curva Precision-Recall')
ax2.legend()
ax2.grid(True, alpha=0.3)

ax3 = axes[1, 0]
ax3.hist(y_pred[y_true == 0], bins=50, alpha=0.5, label='Benignos', color='blue')
ax3.hist(y_pred[y_true == 1], bins=50, alpha=0.5, label='Melanomas', color='red')
ax3.axvline(optimal_threshold, color='black', linestyle='--', lw=2, 
           label=f'Umbral = {optimal_threshold:.3f}')
ax3.set_xlabel('Probabilidad Predicha')
ax3.set_ylabel('Frecuencia')
ax3.set_title('Distribución de Predicciones')
ax3.legend()
ax3.grid(True, alpha=0.3)

ax4 = axes[1, 1]
thresholds_range = np.linspace(0, 1, 100)
sens_list = []
spec_list = []
f1_list = []

for t in thresholds_range:
    y_pred_t = (y_pred >= t).astype(int)
    cm_t = confusion_matrix(y_true, y_pred_t)
    tn, fp, fn, tp = cm_t.ravel()
    
    sens = tp / (tp + fn) if (tp + fn) > 0 else 0
    spec = tn / (tn + fp) if (tn + fp) > 0 else 0
    f1 = 2 * tp / (2 * tp + fp + fn) if (2 * tp + fp + fn) > 0 else 0
    
    sens_list.append(sens)
    spec_list.append(spec)
    f1_list.append(f1)

ax4.plot(thresholds_range, sens_list, 'r-', label='Sensibilidad')
ax4.plot(thresholds_range, spec_list, 'b-', label='Especificidad')
ax4.plot(thresholds_range, f1_list, 'g-', label='F1-Score')
ax4.axvline(optimal_threshold, color='black', linestyle='--', lw=2)
ax4.set_xlabel('Umbral')
ax4.set_ylabel('Valor de Métrica')
ax4.set_title('Métricas vs Umbral')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('model_evaluation_comprehensive.png', dpi=300, bbox_inches='tight')
plt.show()

## 11. Conversión a TensorFlow Lite

In [ ]:
model.save('melanoma_efficientnet_full.h5')
print(f"Modelo completo guardado: {os.path.getsize('melanoma_efficientnet_full.h5') / 1024 / 1024:.1f} MB")

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('melanoma_efficientnet_float32.tflite', 'wb') as f:
    f.write(tflite_model)

print(f"\nTFLite Float32: {os.path.getsize('melanoma_efficientnet_float32.tflite') / 1024 / 1024:.1f} MB")

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_dynamic_model = converter.convert()

with open('melanoma_efficientnet_dynamic.tflite', 'wb') as f:
    f.write(tflite_dynamic_model)

print(f"TFLite Dynamic Quant: {os.path.getsize('melanoma_efficientnet_dynamic.tflite') / 1024 / 1024:.1f} MB")

def representative_dataset():
    for i in range(200):
        batch = next(val_generator)[0]
        yield [batch]

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS_INT8,
    tf.lite.OpsSet.TFLITE_BUILTINS  # Fallback para ops no soportadas
]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.float32  # Mantener output en float para precisión

try:
    tflite_int8_model = converter.convert()
    
    with open('melanoma_efficientnet_int8.tflite', 'wb') as f:
        f.write(tflite_int8_model)
    
    print(f"TFLite INT8: {os.path.getsize('melanoma_efficientnet_int8.tflite') / 1024 / 1024:.1f} MB")
except Exception as e:
    print(f"No se pudo crear la versión INT8: {e}")

## 12. Validación del Modelo TFLite

In [ ]:
def evaluate_tflite_model(tflite_path, test_generator, max_samples=500):
    interpreter = tf.lite.Interpreter(model_path=tflite_path)
    interpreter.allocate_tensors()
    
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    predictions = []
    y_true = []
    
    test_generator.reset()
    samples_processed = 0
    
    for i in range(len(test_generator)):
        if samples_processed >= max_samples:
            break
            
        batch_images, batch_labels = next(test_generator)
        
        for j in range(len(batch_images)):
            if samples_processed >= max_samples:
                break
                
            input_data = np.expand_dims(batch_images[j], axis=0)
            
            if input_details[0]['dtype'] == np.uint8:
                input_data = (input_data * 255).astype(np.uint8)
            else:
                input_data = input_data.astype(np.float32)
            
            interpreter.set_tensor(input_details[0]['index'], input_data)
            interpreter.invoke()
            
            output_data = interpreter.get_tensor(output_details[0]['index'])
            predictions.append(output_data[0][0])
            y_true.append(batch_labels[j])
            
            samples_processed += 1
    
    y_true = np.array(y_true)
    predictions = np.array(predictions)
    
    auc = roc_auc_score(y_true, predictions)
    y_pred_binary = (predictions >= optimal_threshold).astype(int)
    
    cm = confusion_matrix(y_true, y_pred_binary)
    tn, fp, fn, tp = cm.ravel()
    
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    
    return {
        'auc': auc,
        'sensitivity': sensitivity,
        'specificity': specificity,
        'model_size_mb': os.path.getsize(tflite_path) / 1024 / 1024
    }

# Comparar modelos
print("\nComparación de los Modelos TFLite:")
print("-" * 60)

for model_path, model_name in [
    ('melanoma_efficientnet_float32.tflite', 'Float32'),
    ('melanoma_efficientnet_dynamic.tflite', 'Dynamic Quant'),
    ('melanoma_efficientnet_int8.tflite', 'INT8')
]:
    if os.path.exists(model_path):
        print(f"\nEvaluando {model_name}...")
        metrics = evaluate_tflite_model(model_path, test_generator)
        print(f"  - Tamaño: {metrics['model_size_mb']:.1f} MB")
        print(f"  - AUC: {metrics['auc']:.4f}")
        print(f"  - Sensibilidad: {metrics['sensitivity']:.2%}")
        print(f"  - Especificidad: {metrics['specificity']:.2%}")